## VGG实现

&emsp;&emsp;AlexNet是LeNet的变种，结构不是很清晰，结构很随意。VGG的思想是能不能做到更深更大？

&emsp;&emsp;VGG使用可以重复使用的卷积块来构建深度卷积神经网络。

In [1]:
import torch
import torch.nn as nn

def vgg_block(num_convs, in_channels, out_channels):
    """
    num_convs: 需要多少个卷积层。
    in_channels: 输入的channels。
    out_channels: 输出的channels。
    """
    layers = []
    for _ in range(num_convs):
        layers.append(nn.Conv2d(in_channels, out_channels, kernel_size=3, padding=1))
        layers.append(nn.ReLU())
        in_channels = out_channels
    layers.append(nn.MaxPool2d(kernel_size=2, stride=2))
    return nn.Sequential(*layers)

&emsp;&emsp;VGG需要指定一个架构:

In [2]:
conv_arch = ((1, 64), (1, 128), (2, 256), (2, 512), (2, 512))  # 网络结构。

def vgg(conv_arch):
    """
    """
    conv_blks = []
    in_channels = 1
    for (num_convs, out_channels) in conv_arch:
        conv_blks.append(vgg_block(num_convs, in_channels, out_channels))
        in_channels = out_channels
    return nn.Sequential(*conv_blks, nn.Flatten(),
                        nn.Linear(out_channels * 7 * 7, 4096), nn.ReLU(),
                        nn.Dropout(0.5), nn.Linear(4096, 4096), nn.ReLU(),
                        nn.Dropout(0.5), nn.Linear(4096, 10))
net = vgg(conv_arch)

&emsp;&emsp;观察每个层输出的形状:

In [4]:
X = torch.randn(size=(1, 1, 224, 224))
for blk in net:
    X = blk(X)
    print(blk.__class__.__name__, "Out Put Shape: ", X.shape)

Sequential Out Put Shape:  torch.Size([1, 64, 112, 112])
Sequential Out Put Shape:  torch.Size([1, 128, 56, 56])
Sequential Out Put Shape:  torch.Size([1, 256, 28, 28])
Sequential Out Put Shape:  torch.Size([1, 512, 14, 14])
Sequential Out Put Shape:  torch.Size([1, 512, 7, 7])
Flatten Out Put Shape:  torch.Size([1, 25088])
Linear Out Put Shape:  torch.Size([1, 4096])
ReLU Out Put Shape:  torch.Size([1, 4096])
Dropout Out Put Shape:  torch.Size([1, 4096])
Linear Out Put Shape:  torch.Size([1, 4096])
ReLU Out Put Shape:  torch.Size([1, 4096])
Dropout Out Put Shape:  torch.Size([1, 4096])
Linear Out Put Shape:  torch.Size([1, 10])


&emsp;&emsp;上述网络中卷积层加上全连接层的个数是11个，所以是VGG-11。由于它的计算量比AlexNet计算量大，所以我们构建一个通道数较少的网络。

In [5]:
ratio = 4
small_conv_arch = [(pair[0], pair[1] // ratio) for pair in conv_arch]
net = vgg(small_conv_arch)

In [6]:
X = torch.randn(size=(1, 1, 224, 224))
for blk in net:
    X = blk(X)
    print(blk.__class__.__name__, "Out Put Shape: ", X.shape)

Sequential Out Put Shape:  torch.Size([1, 16, 112, 112])
Sequential Out Put Shape:  torch.Size([1, 32, 56, 56])
Sequential Out Put Shape:  torch.Size([1, 64, 28, 28])
Sequential Out Put Shape:  torch.Size([1, 128, 14, 14])
Sequential Out Put Shape:  torch.Size([1, 128, 7, 7])
Flatten Out Put Shape:  torch.Size([1, 6272])
Linear Out Put Shape:  torch.Size([1, 4096])
ReLU Out Put Shape:  torch.Size([1, 4096])
Dropout Out Put Shape:  torch.Size([1, 4096])
Linear Out Put Shape:  torch.Size([1, 4096])
ReLU Out Put Shape:  torch.Size([1, 4096])
Dropout Out Put Shape:  torch.Size([1, 4096])
Linear Out Put Shape:  torch.Size([1, 10])


In [7]:
import torchvision
from torchvision import transforms
from torch.utils import data

def get_dataloader_workers():
    """Use 4 processes to read the data."""
    return 4

def load_data_fashion_mnist(batch_size, resize=None):
    """Download the Fashion-MNIST dataset and then load it into memory."""
    trans = [transforms.ToTensor()]
    
    if resize:
        trans.insert(0, transforms.Resize(resize))
        
    trans = transforms.Compose(trans)
    mnist_train = torchvision.datasets.FashionMNIST(root="../data", train=True, transform=trans, download=True)
    mnist_test = torchvision.datasets.FashionMNIST(root="../data", train=False, transform=trans, download=True)
    
    return (data.DataLoader(mnist_train, batch_size, shuffle=True, num_workers=get_dataloader_workers()),
            data.DataLoader(mnist_test, batch_size, shuffle=False, num_workers=get_dataloader_workers()))

In [8]:
train_iter, test_iter = load_data_fashion_mnist(batch_size=128, resize=224)

/Users/tinyzqh/miniforge3/lib/python3.9/site-packages/torchvision/datasets/mnist.py:498: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /tmp/pip-req-build-wdh4qigd/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [9]:
class Accumulator:
    """For accumulating sums over `n` variables."""
    def __init__(self, n):
        self.data = [0.0] * n

    def add(self, *args):
        self.data = [a + float(b) for a, b in zip(self.data, args)]

    def reset(self):
        self.data = [0.0] * len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]

In [10]:
argmax = lambda x, *args, **kwargs: x.argmax(*args, **kwargs)
reduce_sum = lambda x, *args, **kwargs: x.sum(*args, **kwargs)
astype = lambda x, *args, **kwargs: x.type(*args, **kwargs)

def accuracy(y_hat, y):
    """Compute the number of correct predictions."""
    if len(y_hat.shape) > 1 and y_hat.shape[1] > 1:
        y_hat = argmax(y_hat, axis=1)
    cmp = astype(y_hat, y.dtype) == y
    return float(reduce_sum(astype(cmp, y.dtype)))

In [11]:
def evaluate_accuracy_gpu(net, data_iter, device=None):
    """使用GPU计算模型在数据集上的精度"""
    if isinstance(net, torch.nn.Module):
        net.eval()
        if not device: # 如果没有指定device, 就用网络参数中的所给定的那个device。
            device = next(iter(net.parameters())).device
    
    metric = Accumulator(2)
    
    for X, y in data_iter:
        if isinstance(X, list):
            X = [x.to(device) for x in X]  # 将数据挪到device上。
        else:
            X = X.to(device)
            
        y = y.to(device)
        
        metric.add(accuracy(net(X), y), y.numel())
        
        return metric[0] / metric[1]

In [12]:
def train(net, train_iter, test_iter, num_epochs, lr, device):
    def init_weights(m):
        if type(m) == nn.Linear or type(m) == nn.Conv2d:
            nn.init.xavier_uniform_(m.weight)
    
    net.apply(init_weights)  # 对net里面的每一个parameter都去run一下init_weights这个函数。
    print("trainng on {}".format(device))
    
    net.to(device)
    
    optimizer = torch.optim.SGD(net.parameters(), lr=lr)
    loss = nn.CrossEntropyLoss()
    
    num_batches = len(train_iter)
    
    for epoch in range(num_epochs):
        metric = Accumulator(3)
        net.train()
        for i, (X, y) in enumerate(train_iter):
            X, y = X.to(device), y.to(device)
            
            y_hat = net(X)
            
            optimizer.zero_grad()
            l = loss(y_hat, y)
            l.backward()
            optimizer.step()
            
            metric.add(l * X.shape[0], accuracy(y_hat, y), X.shape[0])
            
            train_loss = metric[0] / metric[2]
            train_acc = metric[1] / metric[2]
            
            if (i + 1) % (num_batches // 5) == 0 or i == num_batches - 1:
                print("epoch {}, train_loss {}, train_acc {}".format(epoch + (i + 1) / num_batches, 
                                                                  train_loss, train_acc))
    test_acc = evaluate_accuracy_gpu(net, test_iter)
    print(f'loss {train_loss:.3f}, train acc {train_acc:.3f}, 'f'test acc {test_acc:.3f}')

In [13]:
def try_gpu(i=0):
    """Return gpu(i) if exists, otherwise return cpu()."""
    if torch.cuda.device_count() >= i + 1:
        return torch.device(f'cuda:{i}')
    return torch.device('cpu')

In [15]:
lr, num_epochs, batch_size = 0.05, 10, 128
train(net, train_iter, test_iter, num_epochs, lr, try_gpu())

trainng on cpu
epoch 0.19829424307036247, train_loss 2.303045021590366, train_acc 0.09971438172043011
epoch 0.39658848614072495, train_loss 2.303150946094144, train_acc 0.1000084005376344
epoch 0.5948827292110874, train_loss 2.303196661788503, train_acc 0.10016241039426524
epoch 0.7931769722814499, train_loss 2.3031609885154234, train_acc 0.10057543682795698
epoch 0.9914712153518124, train_loss 2.303168814156645, train_acc 0.10057123655913978
epoch 1.0, train_loss 2.303161358642578, train_acc 0.10056666666666667
epoch 1.1982942430703625, train_loss 2.3030748674946446, train_acc 0.10450268817204302


Exception ignored in: <function _MultiProcessingDataLoaderIter.__del__ at 0x13c37f3a0>
Traceback (most recent call last):
  File "/Users/tinyzqh/miniforge3/lib/python3.9/site-packages/torch/utils/data/dataloader.py", line 1323, in __del__
    def __del__(self):
  File "/Users/tinyzqh/miniforge3/lib/python3.9/site-packages/torch/utils/data/_utils/signal_handling.py", line 66, in handler
    _error_if_any_worker_fails()
RuntimeError: DataLoader worker (pid 31936) is killed by signal: Interrupt: 2. 


KeyboardInterrupt: 